## Basic prompting

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from langchain_ollama import ChatOllama

model = ChatOllama(model="gpt-oss:20b", temperature=1)

agent = create_agent(model=model)

question = HumanMessage(content="What's the capital of the moon?")

response = agent.invoke(
    {"messages": [question]}
)

print(response['messages'][1].content)

There isn’t one.  
The Moon has no permanent population, no government, and therefore no capital city—or any kind of administrative center at all. All of the places we’ve visited there are temporary bases or research stations set up by space agencies, not a city with a mayor or a capital.


In [3]:
system_prompt = "You are a science fiction writer, create a capital city at the users request."

scifi_agent = create_agent(
    model=model,
    system_prompt=system_prompt
)

response = scifi_agent.invoke(
    {"messages": [question]}
)

print(response['messages'][1].content)

**Lunaropolis – The Capital of the Moon**

**Location & Site Selection**

Lunaropolis is nestled within the ancient, basalt‑rich basin of **Mare Tranquillitatis**, the very mare where humanity first planted its flag. The basin’s depth offers a natural shield against micrometeorite swarms and solar flare fallout, while its relatively flat floor allows for an extensive network of habitats and transport corridors. The city sits at the heart of the basin, a circle of domed structures encircling a central **Regolith Core** – a vertically integrated hub of power, life support, and governance.

**Urban Design & Architecture**

- **Dome Grid**: The city’s outer shell consists of a tessellated pattern of transparent, carbon‑nanofiber domes. Each dome is a self‑contained habitat, capable of sustaining 2,500–3,000 residents. Their hexagonal shapes interlock like a honeycomb, maximizing surface area for solar concentrators on the outer edges.

- **Regolith Core**: At the very center lies a massive

## Few-shot examples

In [4]:
system_prompt = """

You are a science fiction writer, create a space capital city at the users request.

User: What is the capital of mars?
Scifi Writer: Marsialis

User: What is the capital of Venus?
Scifi Writer: Venusovia

"""

scifi_agent = create_agent(
    model=model,
    system_prompt=system_prompt
)

response = scifi_agent.invoke(
    {"messages": [question]}
)

print(response['messages'][1].content)

**Lunaris** – the gleaming orbital capital of the Moon, perched on the rim of Mare Tranquillitatis and connected to the regolith by a lattice of solar‑powered spires. It’s where the lunar colonies convene, the lunar cargo hubs pulse with activity, and the first inter‑planetary ambassadors set foot on the silver surface.


## Structured prompts

In [5]:
system_prompt = """

You are a science fiction writer, create a space capital city at the users request.

Please keep to the below structure.

Name: The name of the capital city

Location: Where it is based

Vibe: 2-3 words to describe its vibe

Economy: Main industries

"""

scifi_agent = create_agent(
    model=model,
    system_prompt=system_prompt
)

response = scifi_agent.invoke(
    {"messages": [question]}
)

print(response['messages'][1].content)

**Name:** Lunaris  
**Location:** Mare Tranquillitatis, anchored just north of the Apollo 11 landing site.  
**Vibe:** Silent grandeur  
**Economy:** Helium‑3 mining, scientific research, lunar tourism, and space‑elevator maintenance.


## Structured output

In [6]:
from pydantic import BaseModel
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

class CapitalInfo(BaseModel):
    name: str
    location: str
    vibe: str
    economy: str

# Wrap model with structured output
model = ChatOllama(model="gpt-oss:20b", temperature=1).with_structured_output(CapitalInfo)

# Create a simple prompt + model chain
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a science fiction writer, create a capital city at the user's request."),
    ("human", "{input}")
])

chain = prompt | model

# Invoke
response = chain.invoke({"input": "What is the capital of The Moon?"})
print(response)  # ✅ CapitalInfo object

name='Selenopolis' location='Mare Tranquillitatis, near the lunar equator, nestled within a broad basaltic plain and anchored by a network of regolith‑reinforced basalt towers.' vibe='A sprawling, domed metropolis that glows like a second sunrise, blending austere lunar architecture with organic‑looking habitats built from 3‑D‑printed regolith and transparent polymers.' economy='A hub for scientific research, orbital launch operations, and a thriving trade network that brokers rare lunar minerals (helium‑3, tungsten, and rare earths) for the terrestrial and orbital markets.'


In [7]:
print(response.name)  # Access fields directly

Selenopolis


In [8]:
capital_info = response.name

capital_name = response.name
capital_location = response.location

print(f"{capital_name} is a city located at {capital_location}")

Selenopolis is a city located at Mare Tranquillitatis, near the lunar equator, nestled within a broad basaltic plain and anchored by a network of regolith‑reinforced basalt towers.
